In [30]:
import os
import csv
import math

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

In [31]:
# source: https://www.kaggle.com/datasets/danofer/compass
df = pd.read_csv("./data/compas_og.csv")
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event
0,1.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,0,1,0,0,0,-1.0,13/08/2013 6:03,14/08/2013 5:41,13011352CF10A,13/08/2013,NaN,1.0,(F3),Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,14/08/2013,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,0,327,0
1,2.0,miguel hernandez,miguel,hernandez,14/08/2013,Male,18/04/1947,69,Greater than 45,Other,0,1,0,0,0,-1.0,13/08/2013 6:03,14/08/2013 5:41,13011352CF10A,13/08/2013,NaN,1.0,(F3),Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,14/08/2013,Risk of Violence,1,Low,14/08/2013,07/07/2014,14/07/2014,0,334,961,0
2,3.0,michael ryan,michael,ryan,31/12/2014,Male,06/02/1985,31,25 - 45,Caucasian,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,5,Medium,31/12/2014,Risk of Violence,2,Low,31/12/2014,30/12/2014,03/01/2015,0,3,457,0
3,4.0,kevon dixon,kevon,dixon,27/01/2013,Male,22/01/1982,34,25 - 45,African-American,0,3,0,0,0,-1.0,26/01/2013 3:45,05/02/2013 5:36,13001275CF10A,26/01/2013,NaN,1.0,(F3),Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,05/07/2013,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),05/07/2013,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,27/01/2013,Risk of Violence,1,Low,27/01/2013,26/01/2013,05/02/2013,0,9,159,1
4,5.0,ed philo,ed,philo,14/04/2013,Male,14/05/1991,24,Less than 25,African-American,0,4,0,1,4,-1.0,13/04/2013 4:58,14/04/2013 7:02,13005330CF10A,13/04/2013,NaN,1.0,(F3),Possession of Cocaine,1,13011511MM10A,(M1),0.0,16/06/2013,Driving Under The Influence,16/06/2013,16/06/2013,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,14/04/2013,Risk of Violence,3,Low,14/04/2013,16/06/2013,16/06/2013,4,0,63,0


In [32]:
# source: https://www.kaggle.com/code/innocentsky/logistics-regression-compas-recidivism-racial-bias
columns_to_drop = ['id', 'first', 'last', 'in_custody', 'vr_charge_desc', 'r_case_number', 'vr_charge_degree',
                   'c_offense_date', 'c_case_number', 'r_offense_date', 'juv_other_count', 'end', 'event',
                   'screening_date', 'start', 'juv_misd_count', 'juv_fel_count', 'r_days_from_arrest',
                   'r_charge_degree', 'days_b_screening_arrest', 'vr_case_number', 'priors_count.1', 'r_jail_out',
                   'c_arrest_date', 'r_charge_desc', 'r_jail_in', 'violent_recid', 'decile_score.1',
                   'vr_offense_date', 'out_custody']
df.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

df['c_jail_in'] = pd.to_datetime(df['c_jail_in'], dayfirst=True)
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'], dayfirst=True)
df['compas_screening_date'] = pd.to_datetime(df['compas_screening_date'], dayfirst=True)
df['v_screening_date'] = pd.to_datetime(df['v_screening_date'], dayfirst=True)
df['days_in_jail'] = abs((df['c_jail_out'] - df['c_jail_in']).dt.days)

numeric_cols = ['c_days_from_compas', 'v_decile_score',]
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())
categorical_cols = ['c_charge_degree', 'score_text', 'v_score_text', 'c_jail_in', 'c_jail_out', 'c_charge_desc', 'days_in_jail']
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

df = df[['age', 'sex', 'decile_score', 'priors_count', 'race', 'days_in_jail', 'c_days_from_compas', 'is_violent_recid', 'v_decile_score', 'is_recid']]

columns_to_encode = ['age', 'sex', 'race']
le = preprocessing.LabelEncoder()
for column in columns_to_encode:
    df[column] = le.fit_transform(df[column])

df.head()

,age,sex,decile_score,priors_count,race,days_in_jail,c_days_from_compas,is_violent_recid,v_decile_score,is_recid
0,51,1,1,0,5,0.0,1.0,0,1,0
1,51,1,1,0,5,0.0,1.0,0,1,0
2,13,1,5,0,2,0.0,1.0,0,2,-1
3,16,1,3,0,0,10.0,1.0,1,1,1
4,6,1,4,4,0,1.0,1.0,0,3,1


In [33]:
df.to_csv('./data/compas_processed.csv', index=False)

In [34]:
df.nunique()

age                    65
sex                     2
decile_score           11
priors_count           39
race                    6
days_in_jail          386
c_days_from_compas    657
is_violent_recid        2
v_decile_score         11
is_recid                3
dtype: int64